In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd /mnt/naplm/seonghyeon

/mnt/naplm/seonghyeon


In [3]:
from slickconf import load_config, instantiate
import torch.distributed.checkpoint as dcp
import json
import torch
from infer.llama import Transformer
from infer.model_args import ModelArgs
from halite.utils.probe import Probe

In [4]:
conf = load_config('halite/configs/models/llama/llama3_2_3b.py')

In [7]:
model = instantiate(conf.model)

In [8]:
state_dict = {"model": model.state_dict()}
dcp.load(
    state_dict=state_dict,
    checkpoint_id='/mnt/naplm/seonghyeon/llama/halite/llama3.2-3b/',
)

/home/nsml/.local/lib/python3.11/site-packages/torch/distributed/checkpoint/state_dict_loader.py:142: UserWarning: torch.distributed is unavailable or uninitialized, assuming the intent is to load in a single process.
  warnings.warn(


In [5]:
with open('llama/checkpoints/Llama3.2-3B/params.json', "r") as f:
    params = json.loads(f.read())
model_args: ModelArgs = ModelArgs(
    max_seq_len=8192,
    max_batch_size=16,
    **params,
)
checkpoint = torch.load('llama/checkpoints/Llama3.2-3B/consolidated.00.pth', map_location="cpu", weights_only=True)
model_orig = Transformer(model_args)
model_orig.load_state_dict(checkpoint, strict=True)

<All keys matched successfully>

In [6]:
model_orig = model_orig.to('cuda').to(torch.bfloat16).eval()

In [27]:
model = model.to('cuda').to(torch.bfloat16).eval()
model_orig = model_orig.to('cuda').to(torch.bfloat16).eval()

In [7]:
input_ids = torch.tensor([[1, 2, 3, 4, 5, 6, 7, 8]], device='cuda')
start_pos = 0
position_ids = torch.tensor([[0, 1, 2, 3, 4, 5, 6, 7]], device='cuda')

In [16]:
probe = Probe('llama-debug/halite')

In [17]:
with torch.no_grad():
    with probe:
        out = model(input_ids, position_ids=position_ids)

tensor([[[ 0.0356,  0.0009, -0.0229,  ..., -0.0317,  0.0359, -0.0065],
         [ 0.0009,  0.0068, -0.0610,  ..., -0.0201,  0.0206,  0.0133],
         [ 0.0045,  0.0459, -0.0669,  ...,  0.0486,  0.0190, -0.0203],
         ...,
         [ 0.0278,  0.0013, -0.0620,  ...,  0.0254,  0.0493,  0.0156],
         [-0.0014, -0.0036, -0.0376,  ...,  0.0032,  0.0173,  0.0238],
         [-0.0283,  0.0019, -0.0581,  ..., -0.0183, -0.0032,  0.0242]]],
       device='cuda:0', dtype=torch.bfloat16)
tensor([[[ 0.0118,  0.0088, -0.1660,  ..., -0.0583, -0.0361, -0.0391],
         [ 0.0140, -0.0047, -0.1533,  ..., -0.0889, -0.0232, -0.0126],
         [ 0.0315, -0.0065, -0.1660,  ...,  0.0160, -0.0283, -0.0781],
         ...,
         [ 0.0144, -0.0742, -0.1426,  ..., -0.0057,  0.0354,  0.0454],
         [ 0.0068, -0.0430, -0.0654,  ..., -0.0554, -0.0073,  0.0330],
         [-0.0312, -0.0297, -0.0264,  ..., -0.0015, -0.0371,  0.0188]]],
       device='cuda:0', dtype=torch.bfloat16)
tensor([[[-0.0532,  0.25

In [8]:
probe2 = Probe('llama-debug/orig/log')

In [9]:
with torch.no_grad():
    with probe2:
        out_orig = model_orig(input_ids, start_pos)

tensor([[[ 0.0356,  0.0009, -0.0229,  ..., -0.0317,  0.0359, -0.0065],
         [ 0.0009,  0.0068, -0.0610,  ..., -0.0201,  0.0206,  0.0133],
         [ 0.0045,  0.0459, -0.0669,  ...,  0.0486,  0.0190, -0.0203],
         ...,
         [ 0.0278,  0.0013, -0.0620,  ...,  0.0254,  0.0493,  0.0156],
         [-0.0014, -0.0036, -0.0376,  ...,  0.0032,  0.0173,  0.0238],
         [-0.0283,  0.0019, -0.0581,  ..., -0.0183, -0.0032,  0.0242]]],
       device='cuda:0', dtype=torch.bfloat16)
tensor([[[ 0.0118,  0.0088, -0.1660,  ..., -0.0583, -0.0361, -0.0391],
         [ 0.0140, -0.0047, -0.1533,  ..., -0.0889, -0.0232, -0.0126],
         [ 0.0315, -0.0065, -0.1660,  ...,  0.0160, -0.0283, -0.0781],
         ...,
         [ 0.0144, -0.0742, -0.1426,  ..., -0.0057,  0.0354,  0.0454],
         [ 0.0068, -0.0430, -0.0654,  ..., -0.0554, -0.0073,  0.0330],
         [-0.0312, -0.0297, -0.0264,  ..., -0.0015, -0.0371,  0.0188]]],
       device='cuda:0', dtype=torch.bfloat16)
tensor([[[-0.0532,  0.25

In [32]:
key_orig = 'Transformer.layers.20.feed_forward.w2__out'
key = 'TransformerDecoder.blocks.20.ff.module.linear_out__out'
wo_in_orig = torch.load(f'llama-debug/orig/tensors/v4/{key_orig}.pt', weights_only=True)
wo_in = torch.load(f'llama-debug/tensors/v0/{key}.pt', weights_only=True)

In [33]:
(wo_in - wo_in_orig).abs().max()

tensor(0.0059, device='cuda:0', dtype=torch.bfloat16)

In [26]:
wo_in_orig

tensor([[ 0.0171,  0.0269, -0.0172,  ..., -0.0222, -0.0461,  0.0041],
        [ 0.0166, -0.0889, -0.0042,  ..., -0.0583, -0.0884, -0.0298],
        [-0.0306, -0.0165, -0.0583,  ..., -0.0435, -0.0840,  0.0106],
        ...,
        [-0.0286,  0.0099, -0.0488,  ..., -0.0581, -0.0942, -0.0155],
        [-0.0047, -0.0471, -0.0386,  ..., -0.0664, -0.0781,  0.0035],
        [-0.0261, -0.0908, -0.0952,  ..., -0.0344,  0.0737,  0.0625]],
       device='cuda:0', dtype=torch.bfloat16)

In [27]:
wo_in_orig

tensor([[ 0.0171,  0.0269, -0.0172,  ..., -0.0222, -0.0461,  0.0041],
        [ 0.0166, -0.0889, -0.0042,  ..., -0.0583, -0.0884, -0.0298],
        [-0.0306, -0.0165, -0.0583,  ..., -0.0435, -0.0840,  0.0106],
        ...,
        [-0.0286,  0.0099, -0.0488,  ..., -0.0581, -0.0942, -0.0155],
        [-0.0047, -0.0471, -0.0386,  ..., -0.0664, -0.0781,  0.0035],
        [-0.0261, -0.0908, -0.0952,  ..., -0.0344,  0.0737,  0.0625]],
       device='cuda:0', dtype=torch.bfloat16)

In [20]:
out_orig

tensor([[[ 3.5469,  3.9219,  3.9688,  ..., -4.6875, -4.6875, -4.6875],
         [ 6.5625,  7.0000,  4.7188,  ..., -4.0000, -4.0000, -4.0000],
         [ 9.3125,  8.3125, 11.0625,  ..., -4.2188, -4.2188, -4.2188],
         ...,
         [10.1250,  1.7266,  7.8438,  ..., -2.9688, -2.9688, -2.9688],
         [ 3.5781,  0.5000,  7.2188,  ..., -3.5781, -3.5781, -3.5781],
         [ 1.6875, -2.6406,  3.9375,  ..., -2.4219, -2.4219, -2.4219]]],
       device='cuda:0')

In [51]:
out.logits

tensor([[[ 3.5469,  3.9219,  3.9688,  ..., -4.6875, -4.6875, -4.6875],
         [ 6.5938,  7.0312,  4.7188,  ..., -4.0000, -4.0000, -4.0000],
         [ 9.3750,  8.3750, 11.0625,  ..., -4.2188, -4.2188, -4.2188],
         ...,
         [10.1250,  1.7109,  7.8750,  ..., -2.9844, -2.9844, -2.9844],
         [ 3.5781,  0.5781,  7.2500,  ..., -3.6250, -3.6250, -3.6250],
         [ 1.6875, -2.6094,  3.9375,  ..., -2.4531, -2.4531, -2.4531]]],
       device='cuda:0', dtype=torch.bfloat16)

In [41]:
out.logits

tensor([[[ 3.5469,  3.9219,  3.9688,  ..., -4.6875, -4.6875, -4.6875],
         [ 6.5625,  7.0000,  4.7188,  ..., -4.0000, -4.0000, -4.0000],
         [ 9.3125,  8.3125, 11.0625,  ..., -4.2188, -4.2188, -4.2188],
         ...,
         [10.1250,  1.7266,  7.8438,  ..., -2.9688, -2.9688, -2.9688],
         [ 3.5781,  0.5000,  7.2188,  ..., -3.5781, -3.5781, -3.5781],
         [ 1.7188, -2.6562,  3.9375,  ..., -2.4062, -2.4062, -2.4062]]],
       device='cuda:0', dtype=torch.bfloat16)

In [49]:
(out.logits - out_orig).abs().max()

tensor(0.2188, device='cuda:0')

In [36]:
(out.logits - out_orig).abs().max()

tensor(0.0938, device='cuda:0')

In [42]:
!rm /mnt/naplm/seonghyeon/model.pt /mnt/naplm/seonghyeon/model_orig.pt

In [45]:
qkv = torch.load('/mnt/naplm/seonghyeon/model.pt')
qkv_orig = torch.load('/mnt/naplm/seonghyeon/model_orig.pt')

/tmp/ipykernel_42590/4248749713.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  qkv = torch.load('/mnt/naplm/seonghyeon/model.pt')
/tmp/ipykernel_42590/4248749713.py:2: 

In [47]:
(qkv[0] - qkv_orig[0]).abs().max()

tensor(0., device='cuda:0', dtype=torch.bfloat16)

In [48]:
(qkv[1] - qkv_orig[1]).abs().max()

tensor(0., device='cuda:0', dtype=torch.bfloat16)

In [50]:
(qkv[3].float() - qkv_orig[3].float()).abs().max()

tensor(0., device='cuda:0')

In [115]:
qkv_orig[0].shape

torch.Size([1, 8, 24, 128])

In [53]:
from flash_attn import flash_attn_func

In [126]:
qkv[0].shape, qkv_orig[0].shape

(torch.Size([1, 8, 24, 128]), torch.Size([1, 8, 24, 128]))

In [143]:
(qkv[1].reshape(1, 8, 8, 2, 64).transpose(3, 4).reshape(1, 8, 8, 128) - qkv_orig[1]).abs().max()

tensor(0.0312, device='cuda:0', dtype=torch.bfloat16)

In [139]:
qkv_orig[0][0, 0, 10]

tensor([ 0.3652, -0.3867, -0.5703,  0.7031,  0.4648,  0.4414,  0.6367,  0.4336,
        -0.2734,  0.6367,  0.0339, -1.0859,  1.7500, -0.8594,  0.1079, -1.2188,
        -0.4395,  2.4062,  0.3301,  0.2559,  0.0991, -2.0625,  0.6836, -0.6289,
         0.4043,  0.2852, -1.0234, -0.2949,  1.5312,  1.2422, -1.4531,  0.0767,
        -0.4473, -0.6406, -1.9609,  0.9297,  1.0859,  1.3906,  1.0078,  0.6641,
         0.7266, -0.2256,  1.9531, -2.0781,  0.3379,  0.8984, -2.0781,  2.0469,
         0.0286,  0.8281,  0.5312,  0.9648, -1.5781, -3.1250,  0.4980, -0.6055,
         0.6016,  0.7188,  2.2344,  0.6992, -1.0938, -0.5469, -0.4766, -1.6016,
         1.5625,  0.6680, -2.1094,  0.4961, -0.0330,  2.0469, -0.3105, -1.4297,
        -0.0222, -1.7188, -1.5234, -0.6445,  0.1377,  5.1250,  0.1904, -4.0000,
         0.5352, -2.2500,  1.7969,  0.0894, -0.3340,  1.0078, -0.7148, -2.2812,
        -1.6250, -1.4922, -0.6758,  2.0469,  0.5898, -2.4688, -0.0952,  2.6094,
         1.8672, -0.6016,  0.8594,  2.01

In [167]:
(model.blocks['0'].self_attention.module.qkv.weight[:3072].reshape(24, 2, 64, -1).transpose(1, 2).reshape(3072, 3072) - model_orig.layers[0].attention.wq.weight).abs().max()

tensor(0., device='cuda:0', dtype=torch.bfloat16, grad_fn=<MaxBackward1>)

In [233]:
model.blocks['0'].self_attention.module.qkv.weight.shape

torch.Size([5120, 3072])

In [37]:
(qkv[0] - qkv_orig[0]).abs().max()

tensor(0., device='cuda:0', dtype=torch.bfloat16)

In [38]:
(qkv[1] - qkv_orig[1]).abs().max()

tensor(23.6250, device='cuda:0', dtype=torch.bfloat16)

In [41]:
(qkv[1].reshape(1, 8, 24, 2, 64).transpose(3, 4).reshape(1, 8, 3072) - qkv_orig[1]).reshape(-1, 128).abs().max(0)

torch.return_types.max(
values=tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0', dtype=torch.bfloat16),
indices=tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
(qkv[1].reshape(1, 8, 24, 2, 64).transpose(3, 4).reshape(1, 8, 3072) - qkv_orig[1]).reshape(-1, 128).abs().max(0)

In [203]:
qkv[1].shape, qkv_orig[1].shape

(torch.Size([1, 8, 3072]), torch.Size([1, 8, 3072]))

In [207]:
8 * 128

1024

In [204]:
8 * 24 * 2 * 64

24576

In [42]:
(qkv[2] - qkv_orig[2]).abs().max()

tensor(18.3750, device='cuda:0', dtype=torch.bfloat16)

In [51]:
(qkv[2].reshape(1, 8, 8, 2, 64).transpose(3, 4).reshape(1, 8, 1024) - qkv_orig[2]).reshape(-1, 128).abs().max(0)

torch.return_types.max(
values=tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0', dtype=torch.bfloat16),
indices=tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [46]:
qkv[2].shape, qkv_orig[2].shape

(torch.Size([1, 8, 1024]), torch.Size([1, 8, 1024]))

In [56]:
q1 = qkv[1].reshape(1, 8, 24, 128)
k1 = qkv[2].reshape(1, 8, 8, 128)
v1 = qkv[3].reshape(1, 8, 8, 128)
q2 = qkv_orig[1].reshape(1, 8, 24, 128)
k2 = qkv_orig[2].reshape(1, 8, 8, 128)
v2 = qkv_orig[3].reshape(1, 8, 8, 128)

In [58]:
(flash_attn_func(q1, k1, v1, causal=True) - flash_attn_func(q2, k2, v2, causal=True)).abs().max()

tensor(0., device='cuda:0', dtype=torch.bfloat16)

In [119]:
x = torch.arange(2 * 4).reshape(2, 1, 4)

In [43]:
model

TransformerDecoder(
  (embedding): TextEmbedding(
    (embed_input_ids): Embedding(128256, 3072)
    (dropout): Dropout(p=0, inplace=False)
  )
  (pos_embed): Llama3RoPE()
  (blocks): ModuleDict(
    (0): TransformerEncoderBlock(
      (self_attention): ResidualBlock(
        (pre_norm): RMSNorm()
        (module): SelfAttention(
          (qkv): Linear(in_features=3072, out_features=5120, bias=False)
          (attention): Attention(
            (attn_drop): Dropout(p=0, inplace=False)
          )
          (out): Linear(in_features=3072, out_features=3072, bias=False)
        )
        (dropout): Dropout(p=0, inplace=False)
      )
      (ff): ResidualBlock(
        (pre_norm): RMSNorm()
        (module): FeedForward(
          (linear1): Linear(in_features=3072, out_features=16384, bias=False)
          (activation): SwiGLU()
          (dropout): Dropout(p=0, inplace=False)
          (linear2): Linear(in_features=8192, out_features=3072, bias=False)
        )
        (dropout): Drop

In [31]:
5 * None

TypeError: unsupported operand type(s) for *: 'int' and 'NoneType'

In [33]:
9216 / 3072

3.0

In [34]:
model_orig

Transformer(
  (tok_embeddings): Embedding(128256, 3072)
  (layers): ModuleList(
    (0-27): 28 x TransformerBlock(
      (attention): Attention(
        (wq): Linear(in_features=3072, out_features=3072, bias=False)
        (wk): Linear(in_features=3072, out_features=1024, bias=False)
        (wv): Linear(in_features=3072, out_features=1024, bias=False)
        (wo): Linear(in_features=3072, out_features=3072, bias=False)
      )
      (feed_forward): FeedForward(
        (w1): Linear(in_features=3072, out_features=8192, bias=False)
        (w2): Linear(in_features=8192, out_features=3072, bias=False)
        (w3): Linear(in_features=3072, out_features=8192, bias=False)
      )
      (attention_norm): RMSNorm()
      (ffn_norm): RMSNorm()
    )
  )
  (norm): RMSNorm()
  (output): Linear(in_features=3072, out_features=128256, bias=False)
)

In [35]:
3072 + 1024 * 2

5120